# data
we start with raw zip files downloaded directly
goal is to output a pandas dataframe with a row for each client (the info from the 5 json files merged into 1 row, key names)
this dataframe will be save as a csv file

In [1]:
import os
import zipfile
import json
import pandas as pd


In [2]:
def extract_and_merge_jsons_2_list(input_folder):
    """
    Extracts and merges JSON files from nested zip files in the input folder.

    Args:
        input_folder (str): Path to the folder containing zip files.

    Returns:
        a list of all merged json files. 
    """

    
    if not os.path.exists(input_folder):
        raise FileNotFoundError(f"The folder {input_folder} does not exist.")
    
    merged_data = []

    # Iterate through all zip files in the input folder
    for zip_file_name in sorted(os.listdir(input_folder)):
        zip_file_path = os.path.join(input_folder, zip_file_name)
        
        if zipfile.is_zipfile(zip_file_path):
            with zipfile.ZipFile(zip_file_path, 'r') as outer_zip:
                # Extract each client zip file
               for client_zip_name in sorted(outer_zip.namelist()):
                    if client_zip_name.endswith('.zip'):
                        with outer_zip.open(client_zip_name) as client_zip_file:
                            with zipfile.ZipFile(client_zip_file) as client_zip:
                                # Create a dictionary to store client data
                                client_data = {}
                                # Extract specific JSON files for the client
                                for json_file_name in client_zip.namelist():
                                    if json_file_name in [
                                        "passport.json",
                                        "client_profile.json",
                                        "client_description.json",
                                        "account_form.json",
                                        "label.json",
                                    ]:
                                        with client_zip.open(json_file_name) as json_file:
                                            json_data = json.load(json_file)
                                            # Use the file name (without .json) as the key
                                            key = os.path.splitext(json_file_name)[0]
                                            client_data[key] = json_data
                                # Add the client data to the merged data if all fields are present
                                if client_data:
                                    merged_data.append(client_data)
    
    return merged_data


# # Example usage
# if __name__ == "__main__":
#     input_folder_path = "input"
#     output_json_path = "preprocessing/all_clients"
#     extract_and_merge_jsons(input_folder_path, output_json_path)

In [ ]:
# check 
def check_extract_and_merge_jsons_2_list():
    input_folder_path = "C:\\Users\\Maex\\Documents\\datathon2025\\ag_new_minimal\\raw_ds"
    result = extract_and_merge_jsons_2_list(input_folder_path)
    print(len(result))
    print(result[0])
    

In [ ]:
# check_extract_and_merge_jsons_2_list()

10000
{'passport': {'first_name': 'Freja', 'middle_name': 'Katrine', 'last_name': 'Christensen', 'gender': 'F', 'country': 'Denmark', 'country_code': 'DNK', 'nationality': 'Danish', 'birth_date': '2002-04-18', 'passport_number': 'UE2130779', 'passport_mrz': ['P<DNKCHRISTENSEN<<FREJA<KATRINE<<<<<<<<<<<<<<', 'UE2130779DNK020418<<<<<<<<<<<<<<<<<<<<<<<<<<<'], 'passport_issue_date': '2017-05-11', 'passport_expiry_date': '2027-05-10'}, 'client_profile': {'name': 'Freja Katrine Christensen', 'address': {'city': 'Aalborg', 'street name': 'Strøget', 'street number': 57, 'postal code': '2044'}, 'country_of_domicile': 'Denmark', 'birth_date': '2002-04-18', 'nationality': 'Danish', 'passport_number': 'UE2130779', 'passport_issue_date': '2017-05-11', 'passport_expiry_date': '2027-05-10', 'gender': 'F', 'phone_number': '53 11 20 42', 'email_address': 'freja.christensen@yousee.dk', 'marital_status': 'single', 'secondary_school': {'name': 'Holstebro Gymnasium', 'graduation_year': 2022}, 'higher_educat

In [34]:
def convert_dtype(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype('string')
    return df

In [35]:
# if applicable, convert strings to float or even int64

def try_convert_str2float(col: pd.Series, threshold: float = 1.0) -> pd.Series:
    """
    Attempt to convert a text column to numeric (int or float).

    Parameters:
      col       : A pandas Series of type string.
      threshold : The fraction of non-null values that must convert successfully
                  to accept the conversion (default 1.0 means all values must convert).

    Returns:
      The converted Series if the conversion rate meets the threshold,
      otherwise the original Series.
    """
    # Attempt to convert all values to numeric.
    converted = pd.to_numeric(col, errors='coerce')

    # Count non-null values in the original and in the converted series.
    original_non_null = col.notna().sum()
    converted_non_null = converted.notna().sum()

    # If the column is entirely null, return as is.
    if original_non_null == 0:
        return col

    conversion_rate = converted_non_null / original_non_null

    if conversion_rate >= threshold:
        # Check if all non-null converted values are integers.
        if converted.dropna().apply(lambda x: float(x).is_integer()).all():
            # Convert to pandas nullable integer type.
            return converted.astype('Int64')
        else:
            return converted
    else:
        # If too many values couldn't be converted, keep the original column.
        return col

In [36]:
def try_mixed_convert_str2float(df):
    for col in df.select_dtypes(include=["string"]).columns:
        df[col] = try_convert_str2float(df[col], threshold=1.0)
    return df
# df now has its text columns converted to numeric types (int or float) when possible.

In [37]:
# if applicable, convert strings to float or even int64

def try_convert_str2float(col: pd.Series, threshold: float = 1.0) -> pd.Series:
    """
    Attempt to convert a text column to numeric (int or float).

    Parameters:
      col       : A pandas Series of type string.
      threshold : The fraction of non-null values that must convert successfully
                  to accept the conversion (default 1.0 means all values must convert).

    Returns:
      The converted Series if the conversion rate meets the threshold,
      otherwise the original Series.
    """
    # Attempt to convert all values to numeric.
    converted = pd.to_numeric(col, errors='coerce')

    # Count non-null values in the original and in the converted series.
    original_non_null = col.notna().sum()
    converted_non_null = converted.notna().sum()

    # If the column is entirely null, return as is.
    if original_non_null == 0:
        return col

    conversion_rate = converted_non_null / original_non_null

    if conversion_rate >= threshold:
        # Check if all non-null converted values are integers.
        if converted.dropna().apply(lambda x: float(x).is_integer()).all():
            # Convert to pandas nullable integer type.
            return converted.astype('Int64')
        else:
            return converted
    else:
        # If too many values couldn't be converted, keep the original column.
        return col

In [38]:
# turn list of json into pandas dataframe
def list_json_2_df(list_json):
    df = pd.json_normalize(list_json)
    
    
    df = df.rename(columns = {'label.label':'label'})
    df['label'] = df['label'].replace({'Accept':1, 'Reject':0})

    df = try_mixed_convert_str2float(convert_dtype(df))

    return df

In [42]:
def check_list_json_2_df():
    input_folder_path = "C:\\Users\\Maex\\Documents\\datathon2025\\ag_new_minimal\\raw_ds"
    
    result = list_json_2_df(extract_and_merge_jsons_2_list(input_folder_path))

    # print(result['label.label'])
    

    return result

In [43]:
check_list_json_2_df()

C:\Users\Maex\AppData\Local\Temp\ipykernel_17028\2406927069.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'Accept':1, 'Reject':0})


,passport.first_name,passport.middle_name,passport.last_name,passport.gender,passport.country,passport.country_code,passport.nationality,passport.birth_date,passport.passport_number,passport.passport_mrz,...,account_form.country_of_domicile,account_form.phone_number,account_form.email_address,client_description.Summary Note,client_description.Family Background,client_description.Education Background,client_description.Occupation History,client_description.Wealth Summary,client_description.Client Summary,label
0,Freja,Katrine,Christensen,F,Denmark,DNK,Danish,2002-04-18,UE2130779,['P<DNKCHRISTENSEN<<FREJA<KATRINE<<<<<<<<<<<<<...,...,Denmark,53 11 20 42,freja.christensen@yousee.dk,Freja Katrine Christensen and the RM were intr...,Freja Katrine Christensen is currently single....,Freja obtained her secondary school diploma fr...,Freja Katrine Christensen is a 22 year old and...,She did not have any savings to invest in fina...,The RM is excited to help Freja navigate the c...,1
1,Thomas,Laurent,Lemaître,M,France,FRA,French,1990-02-07,OT9354543,['P<FRALEMAÎTRE<<THOMAS<LAURENT<<<<<<<<<<<<<<<...,...,France,++4903 52 25 79 49,thomas.lemaitre@yahoo.com,The RM first encountered Thomas Laurent Lemaît...,Thomas Laurent Lemaître is currently divorced....,Thomas graduated from Lycée International de L...,Thomas Laurent Lemaître is a 35 year old and c...,He managed to save approximately 80000 EUR fro...,Given the client's impressive career history a...,0
2,Gauthier,,Bernard,M,France,FRA,French,1974-05-31,XV2857876,['P<FRABERNARD<<GAUTHIER<<<<<<<<<<<<<<<<<<<<<<...,...,France,06 85 81 21 11,gauthier.bernard@numericable.fr,Gauthier Bernard and the RM met at a financial...,Gauthier Bernard is currently divorced. He doe...,"In 1992, Gauthier graduated from Lycée Gustave...",Gauthier Bernard is a 50 year old Biotech Star...,"Throughout his career, he saved 360000 EUR, in...","Based on the information provided, we are exci...",1
3,Louna,Ève,Bertrand,F,France,FRA,French,1977-12-25,KI8826467,['P<FRABERTRAND<<LOUNA<ÈVE<<<<<<<<<<<<<<<<<<<<...,...,France,+33 04 18 03 86 04,louna.bertrand@hotmail.com,The RM has known Louna Ève Bertrand since chil...,Louna Ève Bertrand is currently divorced. Her ...,Louna completed her secondary education at Lyc...,"Having worked for over 24 years, Louna Ève Ber...","Throughout her career, she saved 350000 EUR, i...","In summary, Louna has demonstrated a strong wo...",1
4,Britt,Daantje,Van Dijk,F,Netherlands,NLD,Dutch,1982-03-13,XA0813292,['P<NLDVAN DIJK<<BRITT<DAANTJE<<<<<<<<<<<<<<<<...,...,Netherlands,+31 06 92689079,britt.vandijk@yahoo.com,Britt Daantje Van Dijk and the RM crossed path...,Britt Daantje Van Dijk and Schipper have been ...,Britt received her secondary school diploma fr...,"Having worked for over 20 years, Britt Daantje...",She managed to save approximately 210000 EUR f...,"In light of the above, we are optimistic about...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Schipper,Wagemakers,De Jong,M,Netherlands,NLD,Dutch,1997-10-11,OQ9279820,['P<NLDDE JONG<<SCHIPPER<WAGEMAKERS<<<<<<<<<<<...,...,Netherlands,+31 06 47249120,schipper.dejong@icloud.com,Schipper Wagemakers De Jong and the RM crossed...,Schipper Wagemakers De Jong and Luna tied the ...,Schipper received his secondary school diploma...,Schipper Wagemakers De Jong is a 27 year old A...,"During his work, he focused on financial growt...","In summary, Schipper has demonstrated a strong...",1
9996,Jasmijn,Daniëlle,Veenhoven,F,Netherlands,NLD,Dutch,1967-10-04,OW8331312,['P<NLDVEENHOVEN<<JASMIJN<DANIËLLE<<<<<<<<<<<<...,...,Netherlands,+31 06 26816963,jasmijn.veenhoven@kpnmail.nl,Jasmijn Daniëlle Veenhoven and the RM were int...,Jasmijn Daniëlle Veenhoven and Smeets tied the...,Jasmijn received her secondary school diploma ...,Jasmijn Daniëlle Veenhoven is a 57 year old an...,"While working, she saved 1760000 EUR, which sh...","In conclusion, we are confident in the client'...",0
9997,Steiner,Maier,Haider

In [44]:
def create_data():
    input_folder_path = "C:\\Users\\Maex\\Documents\\datathon2025\\ag_new_minimal\\raw_ds"
    return list_json_2_df(extract_and_merge_jsons_2_list(input_folder_path))

In [45]:
ds = create_data()

C:\Users\Maex\AppData\Local\Temp\ipykernel_17028\2406927069.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'Accept':1, 'Reject':0})


# Train 


In [46]:
import autogluon
label = 'label'

In [47]:
from autogluon.tabular import TabularPredictor

In [48]:
train_ds = ds.sample(frac=0.8, random_state=71)
test_ds = ds.drop(train_ds.index)


In [49]:
predictor = TabularPredictor(label=label).fit(train_ds)

No path specified. Models will be saved in: "AutogluonModels\ag-20250406_022427"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
Memory Avail:       4.45 GB / 15.78 GB (28.2%)
Disk Space Avail:   241.61 GB / 930.90 GB (26.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         